In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
import glob
import cv2
from PIL import Image

In [2]:
number_of_files = 0
# number_of_files = 59925
destination_dir = '/home/adauton/Desktop/dataset_autopilot_1hour_noise_norain/'
source_dir = '/home/adauton/Documents/Mestrado/Tese/PythonAPI/dataset_autopilot_1hour_noise_norain/'

for i in os.listdir(source_dir):
    number_of_files += 1

print(number_of_files)

107925


In [13]:
data = h5py.File(destination_dir + 'dataset_autopilot_1hour_noise_norain.h5', 'w')
data.create_dataset('images', shape=(number_of_files, 180, 300, 3), maxshape=(None, 180, 300, 3), dtype='uint8',chunks=True)
data.create_dataset('input_2', shape=(number_of_files, 6), maxshape=(None, 6), chunks=True)
data.create_dataset('labels', shape=(number_of_files, 3), maxshape=(None, 3), chunks=True)

<HDF5 dataset "labels": shape (107925, 3), type "<f4">

In [14]:
IMAGES = []
INPUT_2 = []
LABELS = []

counter = 0
start_index = 0
exceptions = 0

for index, file_name in enumerate(os.listdir(source_dir)):
    try:
        np_data = np.load(source_dir + file_name)
    except:
        exceptions += 1
        print(f'something wrong with {file_name} - counter: {exceptions}')
        continue

    counter += 1

    hlc = np.zeros((4,), dtype='uint8')
    command = np_data['y'][3]
    if command == 'Left':
        hlc[0] = 1
    elif command == 'Right':
        hlc[1] = 1
    elif command == 'Straight':
        hlc[2] = 1
    else:
        hlc[3] = 1
    speed_and_speed_limit = np_data['y'][4:6].astype(float)

    image = np_data['x']
    input_2 = np.concatenate((speed_and_speed_limit, hlc))
    label = np_data['y'][:3].astype(float)

    IMAGES.append(image)
    INPUT_2.append(input_2)
    LABELS.append(label)

    if counter % 10000 == 0:
        final_index = counter
        data['images'][start_index:final_index] = IMAGES
        data['input_2'][start_index:final_index] = INPUT_2
        data['labels'][start_index:final_index] = LABELS

        start_index = final_index

        IMAGES = []
        INPUT_2 = []
        LABELS = []

        print(f'first {counter} elements computed')

data['images'][start_index:final_index] = IMAGES
data['input_2'][start_index:final_index] = INPUT_2
data['labels'][start_index:final_index] = LABELS

start_index = final_index

print(f'first {counter} elements computed')

IMAGES = []
INPUT_2 = []
LABELS = []


first 10000 elements computed
first 20000 elements computed
first 30000 elements computed
first 40000 elements computed
first 50000 elements computed
first 60000 elements computed
first 70000 elements computed
first 80000 elements computed
first 90000 elements computed
first 100000 elements computed
first 107925 elements computed


In [15]:
data.close()

In [8]:
for f in os.listdir(source_dir):
    os.remove(os.path.join(source_dir, f))